#### Introduction and relevant notes

In [5]:
A = rand(1:4,3,3)

3×3 Matrix{Int64}:
 4  1  3
 4  4  2
 3  3  1

In [6]:
B = A
C = copy(A)
[B C]

3×6 Matrix{Int64}:
 4  1  3  4  1  3
 4  4  2  4  4  2
 3  3  1  3  3  1

In [7]:
# Watch out!
A[1] = 17
[B C] #= B and A are references to the same memory. 
Basically, when doing B=A, we are just declaring a pointer to the memory location of A. =+

3×6 Matrix{Int64}:
 17  1  3  4  1  3
  4  4  2  4  4  2
  3  3  1  3  3  1

In [8]:
x = ones(3)

3-element Vector{Float64}:
 1.0
 1.0
 1.0

________
#### Matrix multiplication

In [9]:
b = A*x

3-element Vector{Float64}:
 21.0
 10.0
  7.0

#### Transposition

    - A' is the conjugate transpose
    - A.' is just the transpose

In [10]:
Asym = A  + A'

3×3 Matrix{Int64}:
 34  5  6
  5  8  5
  6  5  2

#### Transposed multiplication

In [11]:
Apd = A'A

3×3 Matrix{Int64}:
 314  42  62
  42  26  14
  62  14  14

#### Solving linear system

The problem $\mathbf{A} \mathbf{x} = \mathbf{b}$ is solved by the following procedure:

In [12]:
A\b

3-element Vector{Float64}:
 1.0000000000000002
 1.0000000000000002
 0.9999999999999991

##### Overdetermined systems

When A is tall the \function calculates the least squares solution.

In [13]:
Atall = A[:,1:2]
display(Atall)
Atall\b

3×2 Matrix{Int64}:
 17  1
  4  4
  3  3

2-element Vector{Float64}:
 1.1599999999999993
 1.28

The \ function also worls for rank deficient least squares problem. If it is not unique, Julia will return the solution with smallest norm

In [14]:
A = randn(3,3)

3×3 Matrix{Float64}:
 1.10426   -0.219441  -0.823718
 0.167011   0.789494  -0.599085
 0.64795    1.12375    0.282494

In [15]:
# Creating a singular matrix
[A[:,1] A[:,1]]\b

2-element Vector{Float64}:
 8.81616825924116
 8.816168259241161

##### Underdetermined systems

When A is short the \function returns the minimum norm solution

In [16]:
Ashort = A[1:2,:]
display(Ashort)
Ashort\b[1:2]

2×3 Matrix{Float64}:
 1.10426   -0.219441  -0.823718
 0.167011   0.789494  -0.599085

3-element Vector{Float64}:
  11.29794721227096
   2.016197149101197
 -10.885508953909952

________

### Julia is FAST

In [19]:
#using Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

In [25]:
a = rand(10^7)

10000000-element Vector{Float64}:
 0.25662726306749883
 0.39253732246047335
 0.9528118240944873
 0.690926586435737
 0.6886397255330314
 0.22650804577212613
 0.025112114432805255
 0.40764185937029873
 0.963092310127253
 0.5657485057770796
 ⋮
 0.5989869721976998
 0.14898483675122143
 0.6459892773582314
 0.7466267552845279
 0.47000499674230656
 0.3325821030789252
 0.364702579383718
 0.4462723513866562
 0.26358266059038327

In [26]:
sum(a)

5.001612618682886e6

In [ ]:
C_code = """
# include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; i++) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname() # Make a temporary file

# compile to a shared library by piping C_code to gcc (works only if gcc is installed)

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." *
Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [23]:
#Pkg.add("PyCall")
using PyCall # Provides a Julia interface to Python

In [ ]:
apy_list = PyCall.array2py(a, 1, 1)

pysum = pybuiltin("sum")

In [ ]:
pysum(a)

In [31]:
#import Pkg; Pkg.add("Conda")
using Conda

numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # Convert to a numpy array2py
py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

   Resolving package versions...


    Updating `C:\Users\diego\.julia\environments\v1.7\Project.toml`
  [8f4d0f93] + Conda v1.7.0
  No Changes to `C:\Users\diego\.julia\environments\v1.7\Manifest.toml`


BenchmarkTools.Trial: 353 samples with 1 evaluation.
 Range (min … max):  10.660 ms … 20.117 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.948 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.170 ms ±  1.726 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▂  ▁  ▁▄▆ █▁▅▄▄▂▂ ▆▁▁                               
  ▃▃▄▃▃▃▄▅▆█▇▃██▆███▃█████████████▆█▆▅▆▆▄▃▅▄▄▅▁▃▃▇▆▃▃▁▁▁▃▃▃▃▃ ▄
  10.7 ms         Histogram: frequency by time          19 ms <

 Memory estimate: 32 bytes, allocs estimate: 2.

In [33]:
numpy_sum(apy_numpy) # python thing

5.001612618682879e6

In [34]:
numpy_sum(apy_numpy) ≈ sum(a)

true

In [35]:
minimum(py_numpy_bench.times) / 1e6

10.6604

In [37]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 895 samples with 1 evaluation.
 Range (min … max):  3.947 ms … 11.507 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.225 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.566 ms ±  1.057 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

        ▃█▅▄ ▂▁ ▂▁                                            
  ▂▁▁▃▄▅███████████▇█▇▇▇▄▄▅█▅▄▄▄▅▅▄▄▄▃▄▅▅▅▆▄▅▄▅▄▄▃▃▄▃▃▃▁▂▂▃▃ ▄
  3.95 ms        Histogram: frequency by time         8.3 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [38]:
minimum(j_bench.times) / 1e6

3.947